In [0]:
# get the file name from the adf
fileName = dbutils.widgets.get('fileName')

# fileName = 'Product.csv'
fileNameWithoutExt = fileName.split('.')[0]
print(fileNameWithoutExt)

Product

In [0]:
import pyspark.sql.functions as F
#from datetime import datetme as dt

# values based on environemnt and workspace.
sqlDbName = 'apmorgandb'
dbUserName = 'jm'
passwordKey = 'sqldbpasswordkey'
stgAccountSASTokenKey = 'sastoken'
landingFileName =fileName #'Product'  #dbutils.widgets.get('Product')
databricksScopeName ='apmorganscope '
dbServer = 'apmorgandbserver'
dbServerPortNumber ='1433'
storageContainer ='input'
storageAccount='jmapmorganstorage'
landingMountPoint ='/mnt'


In [0]:
dbutils.fs.unmount(mount_point= landingMountPoint)

/mnt has been unmounted.
Out[22]: True

In [0]:
# connect storage account to databricks
# checks if mounted, if not does the mounting - see variable names it uses from above. 
if not any(mount.mountPoint == landingMountPoint for mount in dbutils.fs.mounts()):
    dbutils.fs.mount( source = 'wasbs://{}@{}.blob.core.windows.net'.format(storageContainer, storageAccount), mount_point= landingMountPoint, extra_configs ={'fs.azure.sas.{}.{}.blob.core.windows.net'.format(storageContainer,storageAccount):dbutils.secrets.get(scope = databricksScopeName, key= stgAccountSASTokenKey)})
    print('Mounted the storage account successfully')
else:
    print('Storage account already mounted')

Mounted the storage account successfully

In [0]:
#connect to Azure SQL DB
dbPassword = dbutils.secrets.get(scope = databricksScopeName, key= passwordKey)
serverurl = 'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer, dbServerPortNumber,sqlDbName,dbUserName)
connectionProperties = {
    'password':dbPassword,
    'driver':'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}
df = spark.read.jdbc(url = serverurl, table = 'dbo.FileDetailsFormat', properties= connectionProperties)
display(df)


FileNo FileName ColumnName ColumnDateFormat ColumnIsNull ModifiedDate 1 Product StartDate MM-dd-yyyy true 2012-06-18T22:34:09.000+0000 1 Product EndDate MM/dd/yyyy true 2012-06-18T22:34:09.000+0000 1 Product CreateDate MM/dd/yyyy true 2012-06-18T22:34:09.000+0000 1 Product ModifiedDate MM/dd/yyyy true 2012-06-18T22:34:09.000+0000 2 ProductDescription ModifiedDate MM/dd/yyyy true 2012-06-18T22:34:09.000+0000 2 ProductDescription StartDate MM/dd/yyyy true 2012-06-18T22:34:09.000+0000 2 ProductDescription EndDate MM/dd/yyyy true 2012-06-18T22:34:09.000+0000 3 CustomerDetail CreateDate MM/dd/yyyy true 2012-06-18T22:34:09.000+0000 3 CustomerDetail ActiveDate MM/dd/yyyy true 2012-06-18T22:34:09.000+0000

In [0]:
df1 = spark.read.csv('/mnt/landing/Product.csv', inferSchema=True, header=True)
display(df1)

ProductId Price guid StartDate EndDate CreateDate ModifiedDate NO6034318860278 67 822 14A53EFA-97A9-D852-39F9-E1A366AED2E8 07-27-2021 04/22/2022 12/07/2019 06/16/2020 LU744388398764786734 41 965 07C6BA82-8664-4446-0D52-963BE523C5C7 01-08-2022 03/26/2022 01/01/2020 12/01/2020 IL622786216445251297137 79 352 A4B97E53-2B68-8271-B4C2-F598D1525D92 06-21-2021 04/13/2022 10/23/2019 09/26/2020 HU73530772784633198693907654 93 661 670A2B65-76A4-4B70-B13B-16240274C539 07-29-2021 04/09/2022 02/20/2020 10/19/2020 MT93HZCQ15847141393282243268486 51 231 A2C6ADE3-ABB1-8EC1-5414-8A895C5CA2A4 01-20-2021 03/28/2022 12/22/2019 07/20/2020 ES5071462808326222586356 62 696 E30FC4A9-A2A2-AAB0-58A2-C1382B647D46 09-30-2021 02/26/2022 01/03/2020 06/13/2020 BG44MNMM33864028879242 37 332 E762D650-18D0-3860-A67F-21F0CCE483CD 04-28-2021 06/23/2022 04/12/2020 12/15/2020 EE034518283401745714 85 982 E591D646-4190-4860-8974-D0CC4DE965C6 04-28-2021 12/10/2022 09/25/2019 06/29/2020 IS171916556906574422538815 1 040 C5BAC88D-81A3-3B45-DD0B-6C89EEF896BA 04-27-2021 12/01/2022 09/07/2019 11/06/2020 IE08OFCR35332787214285 82 632 70746AD7-A219-0023-87BF-66F35FC9D77C 01-13-2021 09/01/2022 09/20/2019 11/11/2020 IT341SMIJI45392576391648131 28 250 974C913B-9AA6-76A1-5EF1-272413C48B96 08-10-2021 02/01/2022 12/20/2019 10/28/2020 RS54478177128153438250 3 640 FBD26599-FC99-C1E7-D071-1C13F53E69A7 10-24-2021 12/18/2022 03/16/2020 10/19/2020 DE02807725915261727582 32 639 5621C4D2-7F05-34E5-72D6-0C495B399584 09-29-2021 12/24/2022 03/16/2020 08/08/2020 IL175142853591415592811 66 074 3597B8A7-72B6-D9E8-CAFE-27387C4F3883 07-24-2021 02/18/2022 09/11/2019 09/21/2020 NL65ACGW3711597491 48 032 61B7997B-3E4C-A174-6334-B34563359353 09-27-2021 04/10/2022 02/26/2020 08/01/2020 SK5846425637067253690242 4 307 99677182-2AE2-86AB-355C-8B2D9929C36E 04-23-2021 06/29/2022 03/25/2020 08/08/2020 LT757119267356708661 48 560 FC646D13-2337-4895-040B-834D4A714F9B 04-28-2021 04/02/2022 04/02/2020 09/07/2020 IS033575848530277787917336 35 735 56E9913A-1C13-5C4B-7CA7-B7D5B674A08E 10-17-2021 03/28/2022 04/11/2020 09/03/2020 IT644WWHUI41551393125617462 54 778 4DF9C1B4-365A-1BF1-18C5-20EC2CF48C6B 01-24-2022 02/15/2022 12/08/2019 07/27/2020 PL08196273679325447428092284 82 014 753BCB69-4A59-C1A8-147C-21CC3B99825E 09-26-2021 07/05/2022 02/28/2020 11/02/2020 SI43571181853141544 64 113 E0B3AD31-27BE-EA83-B80D-689D4071A06B 06-26-2021 04/29/2022 05/22/2020 06/21/2020 PK3214772829328054455226 22 200 6E9DD492-BCA8-1A49-0917-B762F63346C6 05-03-2021 09/28/2022 11/08/2019 08/17/2020 LB94498778723178182164225163 27 487 58BF58F4-38C4-D0AD-32E3-C82845746820 06-16-2021 05/08/2022 12/25/2019 12/22/2020 BG97VIEI29664579859946 30 588 2FAA83D4-815D-4673-AF41-D871BE1B1C2D 04-06-2021 07/08/2022 04/09/2020 12/19/2020 SE6486817253112581918341 6 351 879296F4-C68A-A412-8A1D-91C47BC9572A 09-20-2021 08/11/2022 11/19/2019 07/08/2020 GT49448457284435741435266127 10 247 6AD39750-4DA5-44E7-3CDF-6BF1B7CC4D86 02-07-2021 12/16/2022 04/10/2020 11/02/2020 MT61RIUY22012363021323467347744 492 824414A1-FD96-E6B2-CD9B-6AD402F15275 02-02-2021 12/11/2022 05/16/2020 07/28/2020 MT14LNDH29571226356823187493875 13 189 1B97E921-499E-3618-E56D-47E9F8DD9421 08-24-2021 05/10/2022 03/14/2020 10/13/2020 FO7955524667214606 11 169 47CED637-6A2D-821D-A5E2-CC1D5CAE88DC 10-28-2021 03/12/2022 04/02/2020 10/16/2020 AT625565268989564933 89 257 92D1F2C2-C1EA-7B1B-6345-E0C4FB9566BF 10-07-2021 11/25/2022 10/17/2019 12/21/2020 GT78804434582074816337332705 15 372 42818AAA-9FA1-2144-8EA3-68788029FAD6 12-09-2021 06/02/2022 05/10/2020 12/28/2020 IT669WOKHI34253286247718475 51 344 AD88BAC1-B7B1-9179-2ED9-C9C295C22D4A 07-02-2021 10/13/2022 03/27/2020 09/18/2020 PK3666882612559337685371 880 B32FBC38-A873-8258-8534-54CC6692D387 10-04-2021 01/08/2023 11/26/2019 11/23/2020 LV72PRHH4074937636728 60 045 92CDC995-7EEA-AD97-5B66-6BFBCCC47A31 04-20-2021 08/24/2022 10/21/2019 08/14/2020 SM4303321977826254577562642 67 505 ABBEA6CF-37D8-D7AA-187B-54D8DA580365 11-10-2021 08/25/2022 11/30/2019 12/

In [0]:
df1 = spark.read.csv('/mnt/landing/'+fileName, inferSchema=True, header=True)
display(df1)

# Rule
errorFlag=False
errorMessage = ''
totalcount = df1.count()
print(totalcount)
distinctCount = df1.distinct().count()
print(distinctCount)
if distinctCount !=totalcount:
    errorFlag = True
    errorMessage = 'Duplication Found. Rule 1 Failed'
print(errorMessage)
    
# Rule 2
# just filtering on the file name 
df2 = df.filter(df.FileName==fileNameWithoutExt).select('ColumnName','ColumnDateFormat' )
rows = df2.collect()

# only use col name and col date format
for r in rows:
    colName = r[0]
    colFormat =r[1]
    print(colName, colFormat)
    #display(df1.filter(F.to_date(colName, colFormat).isNull() ==True))
    formatCount =df1.filter(F.to_date(colName, colFormat).isNotNull() ==True).count()
    if formatCount == totalcount:
        errorFlag = True
        errorMessage = errorMessage +' DateFormate is incorrect for {} '.format(colName)
    else:
        print('All rows are good for ', colName)
print(errorMessage)

if errorFlag:
    dbutils.fs.mv('/mnt/landing/'+fileName,'/mnt/rejected/'+fileName )
    dbutils.notebook.exit('{"errorFlag": "true", "errorMessage":"'+errorMessage +'"}')
else:
    dbutils.fs.mv('/mnt/landing/'+fileName,'/mnt/staging/'+fileName )
    dbutils.notebook.exit('{"errorFlag": "false", "errorMessage":"No error"}')

{"errorFlag": "true", "errorMessage":" DateFormate is incorrect for StartDate  DateFormate is incorrect for EndDate  DateFormate is incorrect for CreateDate  DateFormate is incorrect for ModifiedDate "}